In [3]:
pip install tweet-preprocessor 


Note: you may need to restart the kernel to use updated packages.


'C:\Users\umang' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import preprocessor as p
import numpy as np 
import pandas as pd 
import emoji
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from sklearn import preprocessing
from keras.layers import  Bidirectional, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors


In [4]:
#data = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
#data2 = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")

In [5]:
df_train = pd.read_csv('C:/Users/umang sharma/Desktop/Final Year Project/Emotion_data/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test = pd.read_csv('C:/Users/umang sharma/Desktop/Final Year Project/Emotion_data/test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')

In [6]:
df_train.head()

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [7]:
misspell_data = pd.read_csv("C:/Users/umang sharma/Desktop/Final Year Project/spell_corrector/aspell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

#Sample of the dict
{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(20)]}

{'nevade': 'Nevada',
 'presbyterian': 'Presbyterian',
 'rsx': 'RSX',
 'Steffen': 'Stephen',
 'susan': 'Susan',
 'abilitey': 'ability',
 'abouy': 'about',
 'absorbtion': 'absorption',
 'accidently': 'accidentally',
 'accomodate': 'accommodate',
 'acommadate': 'accommodate',
 'acord': 'accord',
 'aquantance': 'acquaintance',
 'equire': 'acquire',
 'adultry': 'adultery',
 'aggresive': 'aggressive',
 'alchohol': 'alcohol',
 'alchoholic': 'alcoholic',
 'allieve': 'alive',
 'alright': 'all_right'}

In [8]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

df_train["clean_content"] = df_train.Input.apply(lambda x : misspelled_correction(x))


In [9]:
contractions = pd.read_csv("C:/Users/umang sharma/Desktop/Final Year Project/contractions/contractions.csv")
cont_dic = dict(zip(contractions.Contraction, contractions.Meaning))

In [10]:
def cont_to_meaning(val): 
  
    for x in val.split(): 
        if x in cont_dic.keys(): 
            val = val.replace(x, cont_dic[x]) 
    return val

In [11]:
df_train.clean_content = df_train.clean_content.apply(lambda x : cont_to_meaning(x))

In [12]:
p.set_options(p.OPT.MENTION, p.OPT.URL)
p.clean("Hello people of @nitj #coding 😍 332000 https://github.com/umangsharma")

'Hello people of #coding 😍 332000'

In [13]:
df_train["clean_content"]=df_train.Input.apply(lambda x : p.clean(x))

In [14]:
def punctuation(val): 
  
    punctuations = '''()-[]{};:'"\,<>./@#$%^&_~'''
  
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val

In [15]:
punctuation("test @ #ldfldlf??? !! ")

'test    ldfldlf??? !! '

In [16]:
df_train.clean_content = df_train.clean_content.apply(lambda x : ' '.join(punctuation(emoji.demojize(x)).split()))

In [17]:
def clean_text(val):
    val = misspelled_correction(val)
    val = cont_to_meaning(val)
    val = p.clean(val)
    val = ' '.join(punctuation(emoji.demojize(val)).split())
    
    return val

In [18]:
clean_text("😭 🤔 😟  🎈 🎉 🎪 ")

'loudly crying face thinking face worried face balloon party popper circus tent'

In [19]:
df_train= df_train[df_train.clean_content != ""]

In [20]:
df_train.Sentiment.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: Sentiment, dtype: int64

In [21]:
sent_to_id  = { "joy":1,
                        "sadness":2,"anger":3,"fear":4,"love":5,"surprise":6}

In [22]:
df_train["Sentiment_id"] = df_train['Sentiment'].map(sent_to_id)

In [23]:
df_train

,Input,Sentiment,clean_content,Sentiment_id
0,i didnt feel humiliated,sadness,i didnt feel humiliated,2
1,i can go from feeling so hopeless to so damned...,sadness,i can go from feeling so hopeless to so damned...,2
2,im grabbing a minute to post i feel greedy wrong,anger,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,love,i am ever feeling nostalgic about the fireplac...,5
4,i am feeling grouchy,anger,i am feeling grouchy,3
...,...,...,...,...
15995,i just had a very brief time in the beanbag an...,sadness,i just had a very brief time in the beanbag an...,2
15996,i am now turning and i feel pathetic that i am...,sadness,i am now turning and i feel pathetic that i am...,2
15997,i feel strong and good overall,joy,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,anger,i feel like this was such a rude comment and i...,3


In [24]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_train.Sentiment_id)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_train.clean_content,Y, random_state=1995, test_size=0.2, shuffle=True)


In [26]:
token = text.Tokenizer(num_words=None)
max_len = 160
Epoch = 5

token.fit_on_texts(list(X_train) + list(X_test))
X_train_pad = sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=max_len)
X_test_pad = sequence.pad_sequences(token.texts_to_sequences(X_test), maxlen=max_len)

In [27]:
w_idx = token.word_index

In [28]:
embed_dim = 160
lstm_out = 250

model = Sequential()
model.add(Embedding(len(w_idx) +1 , embed_dim,input_length = X_test_pad.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.core.Dense(6, activation='softmax'))
#adam rmsprop 
model.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 160, 160)          2434080   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 160, 160)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 250)               411000    
_________________________________________________________________
dense (Dense)                (None, 6)                 1506      
Total params: 2,846,586
Trainable params: 2,846,586
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
batch_size = 32

In [30]:
x = model.fit(X_train_pad, y_train, epochs = Epoch, batch_size=batch_size,validation_data=(X_test_pad, y_test))

Epoch 1/5
400/400 [==============================] - 287s 711ms/step - loss: 1.4984 - accuracy: 0.3980 - val_loss: 0.4575 - val_accuracy: 0.8556
Epoch 2/5
400/400 [==============================] - 281s 702ms/step - loss: 0.3133 - accuracy: 0.8976 - val_loss: 0.2286 - val_accuracy: 0.9178
Epoch 3/5
400/400 [==============================] - 272s 680ms/step - loss: 0.1267 - accuracy: 0.9558 - val_loss: 0.1727 - val_accuracy: 0.9325
Epoch 4/5
400/400 [==============================] - 266s 665ms/step - loss: 0.0816 - accuracy: 0.9676 - val_loss: 0.1769 - val_accuracy: 0.9294
Epoch 5/5
400/400 [==============================] - 255s 639ms/step - loss: 0.0469 - accuracy: 0.9850 - val_loss: 0.2620 - val_accuracy: 0.9216


In [31]:
def get_sentiment(model,text):
    text = clean_text(text)
    #tokenize
    twt = token.texts_to_sequences([text])
    twt = sequence.pad_sequences(twt, maxlen=max_len, dtype='int32')
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [32]:
def plot_result(df):
    #colors=['#D50000','#000000','#008EF8','#F5B27B','#EDECEC','#D84A09','#019BBD','#FFD000','#7800A0','#098F45','#807C7C','#85DDE9','#F55E10']
    #fig = go.Figure(data=[go.Pie(labels=df.sentiment,values=df.percentage, hole=.3,textinfo='percent',hoverinfo='percent+label',marker=dict(colors=colors, line=dict(color='#000000', width=2)))])
    #fig.show()
    colors={'joy':'rgb(213,0,0)',
                    'sadness':'rgb(0,142,248)',
                    'anger':'rgb(1,155,189)',
                    'fear':'rgb(9,143,69)',
                    'love':'rgb(128,124,124)',
                    'surprise':'rgb(245,94,16)'}
    col_2={}
    for i in result.sentiment.to_list():
        col_2[i]=colors[i]
    fig = px.pie(df, values='percentage', names='sentiment',color='sentiment',color_discrete_map=col_2,hole=0.3)
    fig.show()

In [34]:
result =get_sentiment(model,"😡🤬😈👿")
plot_result(result)

result =get_sentiment(model,"Raveesh met an accident, i feel very sad for him ")
plot_result(result)

1/1 - 0s


1/1 - 0s


In [35]:
def read_data(file_name):
    with open(file_name,encoding="utf8") as f:
        word_vocab = set() 
        word2vector = {}
        for line in f:
            line_ = line.strip() 
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [36]:
vocab, word_to_idx =read_data("C:/Users/umang sharma/Desktop/Final Year Project/Glove/glove.6B.200d.txt")

Total Words in DataSet: 400000


In [37]:
embedding_matrix = np.zeros((len(w_idx) + 1, 200))
for word, i in w_idx.items():
    embedding_vector = word_to_idx.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [38]:
embed_dim = 200
lstm_out = 250

model_lstm_gwe = Sequential()
model_lstm_gwe.add(Embedding(len(w_idx) +1 , embed_dim,input_length = X_test_pad.shape[1],weights=[embedding_matrix],trainable=False))
model_lstm_gwe.add(SpatialDropout1D(0.2))
model_lstm_gwe.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model_lstm_gwe.add(keras.layers.core.Dense(6, activation='softmax'))
#adam rmsprop 
model_lstm_gwe.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model_lstm_gwe.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 160, 200)          3042600   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 160, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 250)               451000    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1506      
Total params: 3,495,106
Trainable params: 452,506
Non-trainable params: 3,042,600
_________________________________________________________________
None


In [39]:
batch_size = 32

In [40]:
x=model_lstm_gwe.fit(X_train_pad, y_train, epochs = 5, batch_size=batch_size,validation_data=(X_test_pad, y_test))

Epoch 1/5
400/400 [==============================] - 406s 1s/step - loss: 1.4446 - accuracy: 0.4406 - val_loss: 0.8158 - val_accuracy: 0.6928
Epoch 2/5
400/400 [==============================] - 459s 1s/step - loss: 0.7625 - accuracy: 0.7249 - val_loss: 0.4226 - val_accuracy: 0.8478
Epoch 3/5
400/400 [==============================] - 425s 1s/step - loss: 0.4985 - accuracy: 0.8158 - val_loss: 0.3194 - val_accuracy: 0.8728
Epoch 4/5
400/400 [==============================] - 416s 1s/step - loss: 0.3638 - accuracy: 0.8621 - val_loss: 0.2568 - val_accuracy: 0.9013
Epoch 5/5
400/400 [==============================] - 441s 1s/step - loss: 0.2985 - accuracy: 0.8893 - val_loss: 0.2139 - val_accuracy: 0.9169


In [41]:
result =get_sentiment(model_lstm_gwe,"Whats this behaviour😠😠😠😠😠😠😠😠😠😠")
plot_result(result)


1/1 - 1s
